In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import json
import time
from tqdm.notebook import tqdm 

<h4>Start cells here if {state}_geo file has not been FIPS encoded

In [2]:
fips_check = pd.read_csv("data/state_data/geo/geocoded/geo_dc.csv") #- original geocoded file
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1"], axis =1) 
fips_check.head() #4399

,Unnamed: 0,Unnamed: 0.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,Veteran,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long
0,198517,198517,7727377003,4/8/20,353,PPP,"CAVA MEZZE GRILL, LLC",702 H Street NW,WASHINGTON,DC,...,Unanswered,NaN,10116388.89,6/11/21,72.0,Accomodation and Food Services,WASHINGTON DISTRICT OFFICE,"702 H Street NW, WASHINGTON, DC",38.899623,-77.022221
1,198518,198518,8934987202,4/28/20,353,PPP,AMERICAN COLLEGE OF CARDIOLOGY FOUNDATION,"2400 N Street, NW",Washington,DC,...,Unanswered,Y,10115277.78,6/25/21,81.0,Other Services (except Public Administration),WASHINGTON DISTRICT OFFICE,"2400 N Street, NW, Washington, DC",38.907031,-77.052354
2,198519,198519,7310038410,2/11/21,353,PPP,WILLIAM C. SMITH & CO. INC.,1100 New Jersey Ave SE,Washington,DC,...,Unanswered,NaN,NaN,NaN,53.0,Real Estate Rental and Leasing,WASHINGTON DISTRICT OFFICE,"1100 New Jersey Ave SE, Washington, DC",38.877044,-77.003816
3,198520,198520,7754307206,4/28/20,353,PPP,WILEY REIN LLP,1776 K ST,WASHINGTON,DC,...,Unanswered,NaN,9053193.63,11/18/21,54.0,"Professional, Scientific, and Technical Services",WASHINGTON DISTRICT OFFICE,"1776 K ST, WASHINGTON, DC",38.902123,-77.041326
4,198521,198521,9833567004,4/9/20,353,PPP,NATIONAL COMMITTEE FOR QUALITY ASSURANCE,1100 13TH ST NW,WASHINGTON,DC,...,Unanswered,Y,8309510.69,8/3/21,81.0,Other Services (except Public Administration),WASHINGTON DISTRICT OFFICE,"1100 13TH ST NW, WASHINGTON, DC",38.904085,-77.030089
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4394,202911,202911,7366288501,3/5/21,353,PPS,ARRINGTON DIXON & ASSOCIATES INC,2401 Shannon Pl SE,Washington,DC,...,Veteran,NaN,150747.95,9/9/21,56.0,Administrative and Support and Waste Managemen...,WASHINGTON DISTRICT OFFICE,"2401 Shannon Pl SE, Washington, DC",38.863633,-76.993209
4395,202912,202912,7892588508,3/8/21,353,PPS,COLUMBUS RESTAURANTS INC,1205 28th St NW,Washington,DC,...,Non-Veteran,NaN,NaN,NaN,72.0,Accomodation and Food Services,WASHINGTON DISTRICT OFFICE,"1205 28th St NW, Washington, DC",38.905472,-77.056874
4396,202913,202913,8183137301,5/1/20,353,PPP,SAM MEDICAL CENTER OF AMERICA PLLC,2141 K ST NW STE 607,WASHINGTON,DC,...,Unanswered,NaN,13476.46,9/2/21,62.0,Health Care and Social Assistance,WASHINGTON DISTRICT OFFICE,"2141 K ST NW STE 607, WASHINGTON, DC",38.902905,-77.047987
4397,202914,202914,9153308500,3/12/21,353,PPS,BENNETT CAREER INSTITUTE INC,700 Monroe St NE,Washington,DC,...,Non-Veteran,NaN,150780.82,9/23/21,61.0,Educational Services,WASHINGTON DISTRICT OFFICE,"700 Monroe St NE, Washington, DC",38.932611,-76.995942


In [3]:
columns = ['LoanNumber','full_add', 'Lat', 'Long']
fips_check = fips_check[columns]

In [4]:
fips_check_nan = fips_check[fips_check["Lat"].isnull()] # check to make sure no nulls in coords
fips_check_nan #0

,LoanNumber,full_add,Lat,Long


In [5]:
fips_check_notna = fips_check[fips_check["Lat"].notna()] # check to make sure no nulls in coords
fips_check_notna.head()

,LoanNumber,full_add,Lat,Long
0,7727377003,"702 H Street NW, WASHINGTON, DC",38.899623,-77.022221
1,8934987202,"2400 N Street, NW, Washington, DC",38.907031,-77.052354
2,7310038410,"1100 New Jersey Ave SE, Washington, DC",38.877044,-77.003816
3,7754307206,"1776 K ST, WASHINGTON, DC",38.902123,-77.041326
4,9833567004,"1100 13TH ST NW, WASHINGTON, DC",38.904085,-77.030089


<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [ ]:
fips_check_scraped = pd.read_csv("data/state_data/geo/geo_fips/DC_fips_scraped.csv") 
fips_check_scraped.head() #15869

In [7]:
columns = ['LoanNumber','full_add', 'Lat', 'Long', 'FIPS_z']
fips_check_scraped = fips_check_scraped[columns]

In [10]:
#pd.set_option('display.float_format', '{:.0f}'.format)

In [ ]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna.head() 

In [ ]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #7204

<h4>FIPS Script

In [6]:
#if geocode file
lat = fips_check_notna["Lat"].astype(float).astype(str).to_list()
long = fips_check_notna["Long"].astype(float).astype(str).to_list()

#if FIPS_z file 
#lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
#long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [7]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
    
executable_path = {'executable_path': ChromeDriverManager().install()}
driver = Browser('chrome', **executable_path, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('_x_id').clear()
        driver.find_by_id('_y_id').clear()
        driver.find_by_id('_x_id').fill(long[i])
        driver.find_by_id('_y_id').fill(lat[i])
        driver.find_by_id('proceed').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='pl_gov_census_geo_geocoder_domain_GeographyResult')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('11')] #DC
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                notfound.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Driver [/Users/ciaraspencer/.wdm/drivers/chromedriver/mac64/98.0.4758.80/chromedriver] found in cache


coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [10]:
geo_id_list =geo_id 

Length Check

In [11]:
print(len(geo_id_list))

4399


Matching FIPS to DataFrame

In [12]:
#for geocode file
fips_fill = fips_check_notna #confirm pair to notna DataFrame
#if FIPS available
#fips_fill = fips_check_scraped_null.iloc[:7204]

In [13]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

,LoanNumber,full_add,Lat,Long,FIPS_z
0,7727377003,"702 H Street NW, WASHINGTON, DC",38.899623,-77.022221,[110010058025008]
1,8934987202,"2400 N Street, NW, Washington, DC",38.907031,-77.052354,[110010055032003]
2,7310038410,"1100 New Jersey Ave SE, Washington, DC",38.877044,-77.003816,[110010072031004]
3,7754307206,"1776 K ST, WASHINGTON, DC",38.902123,-77.041326,[110010107001015]
4,9833567004,"1100 13TH ST NW, WASHINGTON, DC",38.904085,-77.030089,[110010101003000]
...,...,...,...,...,...
4394,7366288501,"2401 Shannon Pl SE, Washington, DC",38.863633,-76.993209,[110010074011015]
4395,7892588508,"1205 28th St NW, Washington, DC",38.905472,-77.056874,[110010001011003]
4396,8183137301,"2141 K ST NW STE 607, WASHINGTON, DC",38.902905,-77.047987,[110010107001007]
4397,9153308500,"700 Monroe St NE, Washington, DC",38.932611,-76.995942,[110010092011000]


Remove brackets

In [14]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

,LoanNumber,full_add,Lat,Long,FIPS_z
0,7727377003,"702 H Street NW, WASHINGTON, DC",38.899623,-77.022221,110010058025008
1,8934987202,"2400 N Street, NW, Washington, DC",38.907031,-77.052354,110010055032003
2,7310038410,"1100 New Jersey Ave SE, Washington, DC",38.877044,-77.003816,110010072031004
3,7754307206,"1776 K ST, WASHINGTON, DC",38.902123,-77.041326,110010107001015
4,9833567004,"1100 13TH ST NW, WASHINGTON, DC",38.904085,-77.030089,110010101003000
...,...,...,...,...,...
4394,7366288501,"2401 Shannon Pl SE, Washington, DC",38.863633,-76.993209,110010074011015
4395,7892588508,"1205 28th St NW, Washington, DC",38.905472,-77.056874,110010001011003
4396,8183137301,"2141 K ST NW STE 607, WASHINGTON, DC",38.902905,-77.047987,110010107001007
4397,9153308500,"700 Monroe St NE, Washington, DC",38.932611,-76.995942,110010092011000


In [15]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #8

,LoanNumber,full_add,Lat,Long,FIPS_z
474,4189318306,"106 Air Cargo Rd, Washington, DC",38.843642,-77.043856,NaN
1697,5791017300,"1995 W 5TH ST, WASHINGTON, DC",35.574706,-77.085720,NaN
2246,7241587203,"PO Box 92592, WASHINGTON, DC",33.507399,-117.123579,NaN
2463,5693987109,"5101 WISCONSIN AVE UNIT 102, WASHINGTON, DC",38.962670,-77.086822,NaN
2752,2375807102,"4021 University Dr. Suite 200, WASHINGTON, DC",38.844869,-77.305493,NaN
3289,6713817303,"1801 Coral Way Ste 312, WASHINGTON, DC",25.750967,-80.224109,NaN
3350,6590707010,"1 aviation drive, WASHINGTON, DC",38.858413,-77.044853,NaN
3716,8221899006,"2393 Watters Glen Ct.2420 New York Ave. NE, Wa...",38.894416,-77.209191,NaN


Parse for Merge

In [16]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
0,7727377003,110010058025008
1,8934987202,110010055032003
2,7310038410,110010072031004
3,7754307206,110010107001015
4,9833567004,110010101003000
...,...,...
4394,7366288501,110010074011015
4395,7892588508,110010001011003
4396,8183137301,110010107001007
4397,9153308500,110010092011000


Read in base data

In [17]:
#if from geocode file
fips_check = pd.read_csv("data/state_data/geo/geocoded/geo_dc.csv")
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1"], axis =1) 
fips_check

#if from FIPS file
#fips_check = pd.read_csv("data/state_data/geo/geo_fips/DC_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 

#fips_check.head() #9179


,Unnamed: 0,Unnamed: 0.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,Veteran,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long
0,198517,198517,7727377003,4/8/20,353,PPP,"CAVA MEZZE GRILL, LLC",702 H Street NW,WASHINGTON,DC,...,Unanswered,NaN,10116388.89,6/11/21,72.0,Accomodation and Food Services,WASHINGTON DISTRICT OFFICE,"702 H Street NW, WASHINGTON, DC",38.899623,-77.022221
1,198518,198518,8934987202,4/28/20,353,PPP,AMERICAN COLLEGE OF CARDIOLOGY FOUNDATION,"2400 N Street, NW",Washington,DC,...,Unanswered,Y,10115277.78,6/25/21,81.0,Other Services (except Public Administration),WASHINGTON DISTRICT OFFICE,"2400 N Street, NW, Washington, DC",38.907031,-77.052354
2,198519,198519,7310038410,2/11/21,353,PPP,WILLIAM C. SMITH & CO. INC.,1100 New Jersey Ave SE,Washington,DC,...,Unanswered,NaN,NaN,NaN,53.0,Real Estate Rental and Leasing,WASHINGTON DISTRICT OFFICE,"1100 New Jersey Ave SE, Washington, DC",38.877044,-77.003816
3,198520,198520,7754307206,4/28/20,353,PPP,WILEY REIN LLP,1776 K ST,WASHINGTON,DC,...,Unanswered,NaN,9053193.63,11/18/21,54.0,"Professional, Scientific, and Technical Services",WASHINGTON DISTRICT OFFICE,"1776 K ST, WASHINGTON, DC",38.902123,-77.041326
4,198521,198521,9833567004,4/9/20,353,PPP,NATIONAL COMMITTEE FOR QUALITY ASSURANCE,1100 13TH ST NW,WASHINGTON,DC,...,Unanswered,Y,8309510.69,8/3/21,81.0,Other Services (except Public Administration),WASHINGTON DISTRICT OFFICE,"1100 13TH ST NW, WASHINGTON, DC",38.904085,-77.030089
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4394,202911,202911,7366288501,3/5/21,353,PPS,ARRINGTON DIXON & ASSOCIATES INC,2401 Shannon Pl SE,Washington,DC,...,Veteran,NaN,150747.95,9/9/21,56.0,Administrative and Support and Waste Managemen...,WASHINGTON DISTRICT OFFICE,"2401 Shannon Pl SE, Washington, DC",38.863633,-76.993209
4395,202912,202912,7892588508,3/8/21,353,PPS,COLUMBUS RESTAURANTS INC,1205 28th St NW,Washington,DC,...,Non-Veteran,NaN,NaN,NaN,72.0,Accomodation and Food Services,WASHINGTON DISTRICT OFFICE,"1205 28th St NW, Washington, DC",38.905472,-77.056874
4396,202913,202913,8183137301,5/1/20,353,PPP,SAM MEDICAL CENTER OF AMERICA PLLC,2141 K ST NW STE 607,WASHINGTON,DC,...,Unanswered,NaN,13476.46,9/2/21,62.0,Health Care and Social Assistance,WASHINGTON DISTRICT OFFICE,"2141 K ST NW STE 607, WASHINGTON, DC",38.902905,-77.047987
4397,202914,202914,9153308500,3/12/21,353,PPS,BENNETT CAREER INSTITUTE INC,700 Monroe St NE,Washington,DC,...,Non-Veteran,NaN,150780.82,9/23/21,61.0,Educational Services,WASHINGTON DISTRICT OFFICE,"700 Monroe St NE, Washington, DC",38.932611,-76.995942


Merge with MainFrame

In [18]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,Unnamed: 0.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long,FIPS_z
0,198517,198517,7727377003,4/8/20,353,PPP,"CAVA MEZZE GRILL, LLC",702 H Street NW,WASHINGTON,DC,...,NaN,10116388.89,6/11/21,72.0,Accomodation and Food Services,WASHINGTON DISTRICT OFFICE,"702 H Street NW, WASHINGTON, DC",38.899623,-77.022221,110010058025008
1,198518,198518,8934987202,4/28/20,353,PPP,AMERICAN COLLEGE OF CARDIOLOGY FOUNDATION,"2400 N Street, NW",Washington,DC,...,Y,10115277.78,6/25/21,81.0,Other Services (except Public Administration),WASHINGTON DISTRICT OFFICE,"2400 N Street, NW, Washington, DC",38.907031,-77.052354,110010055032003
2,198519,198519,7310038410,2/11/21,353,PPP,WILLIAM C. SMITH & CO. INC.,1100 New Jersey Ave SE,Washington,DC,...,NaN,NaN,NaN,53.0,Real Estate Rental and Leasing,WASHINGTON DISTRICT OFFICE,"1100 New Jersey Ave SE, Washington, DC",38.877044,-77.003816,110010072031004
3,198520,198520,7754307206,4/28/20,353,PPP,WILEY REIN LLP,1776 K ST,WASHINGTON,DC,...,NaN,9053193.63,11/18/21,54.0,"Professional, Scientific, and Technical Services",WASHINGTON DISTRICT OFFICE,"1776 K ST, WASHINGTON, DC",38.902123,-77.041326,110010107001015
4,198521,198521,9833567004,4/9/20,353,PPP,NATIONAL COMMITTEE FOR QUALITY ASSURANCE,1100 13TH ST NW,WASHINGTON,DC,...,Y,8309510.69,8/3/21,81.0,Other Services (except Public Administration),WASHINGTON DISTRICT OFFICE,"1100 13TH ST NW, WASHINGTON, DC",38.904085,-77.030089,110010101003000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4394,202911,202911,7366288501,3/5/21,353,PPS,ARRINGTON DIXON & ASSOCIATES INC,2401 Shannon Pl SE,Washington,DC,...,NaN,150747.95,9/9/21,56.0,Administrative and Support and Waste Managemen...,WASHINGTON DISTRICT OFFICE,"2401 Shannon Pl SE, Washington, DC",38.863633,-76.993209,110010074011015
4395,202912,202912,7892588508,3/8/21,353,PPS,COLUMBUS RESTAURANTS INC,1205 28th St NW,Washington,DC,...,NaN,NaN,NaN,72.0,Accomodation and Food Services,WASHINGTON DISTRICT OFFICE,"1205 28th St NW, Washington, DC",38.905472,-77.056874,110010001011003
4396,202913,202913,8183137301,5/1/20,353,PPP,SAM MEDICAL CENTER OF AMERICA PLLC,2141 K ST NW STE 607,WASHINGTON,DC,...,NaN,13476.46,9/2/21,62.0,Health Care and Social Assistance,WASHINGTON DISTRICT OFFICE,"2141 K ST NW STE 607, WASHINGTON, DC",38.902905,-77.047987,110010107001007
4397,202914,202914,9153308500,3/12/21,353,PPS,BENNETT CAREER INSTITUTE INC,700 Monroe St NE,Washington,DC,...,NaN,150780.82,9/23/21,61.0,Educational Services,WASHINGTON DISTRICT OFFICE,"700 Monroe St NE, Washington, DC",38.932611,-76.995942,110010092011000


<b>Fill FIPS Columns - if FIPS file

In [ ]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge

Drop Extra FIPS Columns

In [ ]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
#fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop

In [ ]:
#fips_merge_drop = fips_merge.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop.loc[fips_merge["FIPS_z"].isnull()] #verify #16 , notna #9163 / 9179

Convert to File

In [19]:
fips_DC = fips_merge
#fips_KY = fips_merge_drop

In [4]:
fips_DC.to_csv("data/state_data/geo/geo_fips/DC_fips_scraped.csv") 

Review & Compare

In [2]:
geo_DC_fips = pd.read_csv("data/state_data/geo/geo_fips/DC_fips_scraped.csv") 
geo_DC_fips

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,...,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long,FIPS_z
0,0,0,0,198517,198517,7727377003,4/8/20,353,PPP,"CAVA MEZZE GRILL, LLC",...,N,10116388.89,6/11/21,72.0,Accomodation and Food Services,WASHINGTON DISTRICT OFFICE,"702 H Street NW, WASHINGTON, DC",38.899623,-77.022221,1.100101e+14
1,1,1,1,198518,198518,8934987202,4/28/20,353,PPP,AMERICAN COLLEGE OF CARDIOLOGY FOUNDATION,...,Y,10115277.78,6/25/21,81.0,Other Services (except Public Administration),WASHINGTON DISTRICT OFFICE,"2400 N Street, NW, Washington, DC",38.907031,-77.052354,1.100101e+14
2,2,2,2,198519,198519,7310038410,2/11/21,353,PPP,WILLIAM C. SMITH & CO. INC.,...,N,NaN,NaN,53.0,Real Estate Rental and Leasing,WASHINGTON DISTRICT OFFICE,"1100 New Jersey Ave SE, Washington, DC",38.877044,-77.003816,1.100101e+14
3,3,3,3,198520,198520,7754307206,4/28/20,353,PPP,WILEY REIN LLP,...,N,9053193.63,11/18/21,54.0,"Professional, Scientific, and Technical Services",WASHINGTON DISTRICT OFFICE,"1776 K ST, WASHINGTON, DC",38.902123,-77.041326,1.100101e+14
4,4,4,4,198521,198521,9833567004,4/9/20,353,PPP,NATIONAL COMMITTEE FOR QUALITY ASSURANCE,...,Y,8309510.69,8/3/21,81.0,Other Services (except Public Administration),WASHINGTON DISTRICT OFFICE,"1100 13TH ST NW, WASHINGTON, DC",38.904085,-77.030089,1.100101e+14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4394,4394,4394,4394,202911,202911,7366288501,3/5/21,353,PPS,ARRINGTON DIXON & ASSOCIATES INC,...,N,150747.95,9/9/21,56.0,Administrative and Support and Waste Managemen...,WASHINGTON DISTRICT OFFICE,"2401 Shannon Pl SE, Washington, DC",38.863633,-76.993209,1.100101e+14
4395,4395,4395,4395,202912,202912,7892588508,3/8/21,353,PPS,COLUMBUS RESTAURANTS INC,...,N,NaN,NaN,72.0,Accomodation and Food Services,WASHINGTON DISTRICT OFFICE,"1205 28th St NW, Washington, DC",38.905472,-77.056874,1.100100e+14
4396,4396,4396,4396,202913,202913,8183137301,5/1/20,353,PPP,SAM MEDICAL CENTER OF AMERICA PLLC,...,N,13476.46,9/2/21,62.0,Health Care and Social Assistance,WASHINGTON DISTRICT OFFICE,"2141 K ST NW STE 607, WASHINGTON, DC",38.902905,-77.047987,1.100101e+14
4397,4397,4397,4397,202914,202914,9153308500,3/12/21,353,PPS,BENNETT CAREER INSTITUTE INC,...,N,150780.82,9/23/21,61.0,Educational Services,WASHINGTON DISTRICT OFFICE,"700 Monroe St NE, Washington, DC",38.932611,-76.995942,1.100101e+14


In [3]:
##### fill in missing Lat, Long, FIPS
geo_DC_fips.loc[geo_DC_fips["FIPS_z"].isnull()] #8

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,...,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long,FIPS_z
474,474,474,474,198991,198991,4189318306,1/23/21,353,PPS,EXPRESS CATERING INC.,...,N,1178560.56,8/18/21,72.0,Accomodation and Food Services,WASHINGTON DISTRICT OFFICE,"106 Air Cargo Rd, Washington, DC",38.843642,-77.043856,NaN
1697,1697,1697,1697,200214,200214,5791017300,4/30/20,353,PPP,INDUS & CONSTR ENTERPRISES,...,N,422263.83,12/28/20,23.0,Construction,WASHINGTON DISTRICT OFFICE,"1995 W 5TH ST, WASHINGTON, DC",35.574706,-77.085720,NaN
2246,2246,2246,2246,200763,200763,7241587203,4/28/20,353,PPP,ULTRA VIOLET EDUCATION FUND,...,Y,320328.50,5/21/21,61.0,Educational Services,WASHINGTON DISTRICT OFFICE,"PO Box 92592, WASHINGTON, DC",33.507399,-117.123579,NaN
2463,2463,2463,2463,200980,200980,5693987109,4/13/20,353,PPP,CHRIST CHILD SOCIETY,...,N,293302.92,10/4/21,81.0,Other Services (except Public Administration),WASHINGTON DISTRICT OFFICE,"5101 WISCONSIN AVE UNIT 102, WASHINGTON, DC",38.962670,-77.086822,NaN
2752,2752,2752,2752,201269,201269,2375807102,4/10/20,353,PPP,WALNUT STREET FINANCE LLC,...,N,NaN,NaN,52.0,Finance and Insurance,WASHINGTON DISTRICT OFFICE,"4021 University Dr. Suite 200, WASHINGTON, DC",38.844869,-77.305493,NaN
3289,3289,3289,3289,201806,201806,6713817303,4/30/20,353,PPP,ALTEX USA CORPORATION,...,N,214604.92,3/31/21,42.0,Wholesale Trade,WASHINGTON DISTRICT OFFICE,"1801 Coral Way Ste 312, WASHINGTON, DC",25.750967,-80.224109,NaN
3350,3350,3350,3350,201867,201867,6590707010,4/7/20,353,PPP,"BMG CONSULTING SERVICES, LLC",...,N,NaN,NaN,54.0,"Professional, Scientific, and Technical Services",WASHINGTON DISTRICT OFFICE,"1 aviation drive, WASHINGTON, DC",38.858413,-77.044853,NaN
3716,3716,3716,3716,202233,202233,8221899006,5/27/21,353,PPS,E&P INC,...,N,NaN,NaN,44.0,Retail Trade,WASHINGTON DISTRICT OFFICE,"2393 Watters Glen Ct.2420 New York Ave. NE, Wa...",38.894416,-77.209191,NaN


In [4]:
geo_DC_fips["BusinessType"] = geo_DC_fips["BusinessType"].replace({'Limited  Liability Company(LLC)': 'Limited Liability Company(LLC)'})
geo_DC_fips.to_csv("data/state_data/geo/geo_fips/DC_fips_scraped.csv")
listbusiness = geo_DC_fips["BusinessType"].value_counts()
listbusiness

Limited Liability Company(LLC)     1434
Corporation                        1279
Non-Profit Organization            1075
Subchapter S Corporation            269
Partnership                          82
Limited Liability Partnership        75
Professional Association             57
Sole Proprietorship                  48
501(c)6 – Non Profit Membership      42
501(c)3 – Non Profit                 15
Cooperative                          13
Non-Profit Childcare Center           9
Not Available                         1
Name: BusinessType, dtype: int64